<a href="https://colab.research.google.com/github/aruaru0/CommonLit-Evaluate-Student-Summaries/blob/main/misspellings2_debertav3L_lgbm_autocorrect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

In [2]:
if is_colab:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
if is_colab:
  !mkdir -p /root/.kaggle
  !cp /content/drive/MyDrive/Kaggle/kaggle.json  /root/.kaggle/
  !chmod 600 /root/.kaggle/kaggle.json

In [4]:
if is_colab:
  !pip install kaggle
  !apt install unzip
  !mkdir input output

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [5]:
if is_colab:
  !kaggle competitions download -c commonlit-evaluate-student-summaries
  !unzip -o commonlit-evaluate-student-summaries.zip -d input/commonlit-evaluate-student-summaries

 95% 1.00M/1.05M [00:00<00:00, 1.08MB/s]
100% 1.05M/1.05M [00:00<00:00, 1.12MB/s]
Archive:  commonlit-evaluate-student-summaries.zip
  inflating: input/commonlit-evaluate-student-summaries/prompts_test.csv  
  inflating: input/commonlit-evaluate-student-summaries/prompts_train.csv  
  inflating: input/commonlit-evaluate-student-summaries/sample_submission.csv  
  inflating: input/commonlit-evaluate-student-summaries/summaries_test.csv  
  inflating: input/commonlit-evaluate-student-summaries/summaries_train.csv  


In [6]:
if is_colab:
  !pip install transformers
  !pip install datasets
  !pip install sentencepiece
  !pip install autocorrect==2.6
  !pip install pyspellchecker==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.6/622.6 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.0-py3-none-any.whl size=622231 sha256=189fbbdf7757e8d9cb40ec4e044672601542555a2b54cfae6161ec526ca2

In [7]:
if is_colab:
  !pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-9vy0mi49
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-9vy0mi49
  Resolved https://github.com/huggingface/accelerate to commit 5ae611118057232f441055f7ef9ba0b0f2b8d533
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.24.0.dev0-py3-none-any.whl size=258568 sha256=12437733acab30fee072376d52c3811f35eb952c51df8611bef95436d5b3a38b
  Stored in directory: /tmp/pip-ephem-wheel-cache-z9fjqwuf/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


# ここでランタイムの再起動が必要！！

In [1]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

if is_colab:
  %cd output

/content/output


In [2]:
if is_colab:
  from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
  model_name = "microsoft/deberta-v3-large"
  # model_name = "roberta-base"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  hidden_dropout_prob=0.007
  attention_probs_dropout_prob=0.007
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  model_dir = "../input/another-bert"
  model.save_pretrained(model_dir)
  tokenizer.save_pretrained(model_dir)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'pooler.dense.weight', 'pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
 # You can confirm which layers have been frozen and see the whole layer struct of the model
 for n, p in model.named_parameters():
    print(n, p.requires_grad)

deberta.embeddings.word_embeddings.weight True
deberta.embeddings.LayerNorm.weight True
deberta.embeddings.LayerNorm.bias True
deberta.encoder.layer.0.attention.self.query_proj.weight True
deberta.encoder.layer.0.attention.self.query_proj.bias True
deberta.encoder.layer.0.attention.self.key_proj.weight True
deberta.encoder.layer.0.attention.self.key_proj.bias True
deberta.encoder.layer.0.attention.self.value_proj.weight True
deberta.encoder.layer.0.attention.self.value_proj.bias True
deberta.encoder.layer.0.attention.output.dense.weight True
deberta.encoder.layer.0.attention.output.dense.bias True
deberta.encoder.layer.0.attention.output.LayerNorm.weight True
deberta.encoder.layer.0.attention.output.LayerNorm.bias True
deberta.encoder.layer.0.intermediate.dense.weight True
deberta.encoder.layer.0.intermediate.dense.bias True
deberta.encoder.layer.0.output.dense.weight True
deberta.encoder.layer.0.output.dense.bias True
deberta.encoder.layer.0.output.LayerNorm.weight True
deberta.encode

In [4]:
def top_half_layer_freeze(model):
    for i in range(0,6,1):
        for n,p in model.named_parameters():
            if f'encoder.layer.{i}.' in n:
                p.requires_grad = False

In [5]:
top_half_layer_freeze(model)

In [6]:
if is_colab:
  import nltk
  nltk.download('stopwords')
  nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Notes
This is a slightly tuned version of @nogawanogawa 's work and I have also converted his messages to english here you can find his notebook here https://www.kaggle.com/code/tsunotsuno/updated-debertav3-lgbm-with-spell-autocorrect please give him kudos for sharing his efforts

### Things I would expect there to be a number of things that will allow this model to preform better outside of just strategy and more data. I would imagine there are a few more tuning parameters that could help this model go a long way.





In this notebook a combonation of Deberta and LGBM is used, pyspellchecker is also used in order to correct some of the spelling mistakes that are discussed in the discussions tab
[Discussion Link](https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/discussion/428941).
[my previous notebook](https://www.kaggle.com/code/tsunotsuno/debertav3-lgbm-with-feature-engineering)
[Discussion Link](https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/discussion/428941).

The primary goal of this notebook is to enhance the overall score by honing in on the issue of "misspellings."

## Main Concept

The Transformers model I'm currently utilizing, Deberta, is pretrained on "correct sentences." However, if I were to train and input it with sentences containing misspellings, Deberta's ability to understand meaning might be compromised.

From a human evaluator's perspective, detecting misspellings would prompt deductions in scores. After discreetly rectifying the misspelled words, I'd proceed to evaluate other textual facets. If we assume the scoring process aligns with this approach, it's conceivable that tallying and **correcting** misspellings before feeding text into Deberta could enable the model to aptly capture features beyond just misspellings.

In this notebook, I will embark on the journey of auto-correcting misspelled words before inputting them into Deberta. The aim is to evaluate the model's performance by distinctly isolating misspellings from other aspects.

### Feature Engineering

I intend to largely retain the same features as before:

- Text Length
- Length Ratio
- Word Overlap
- N-grams Co-occurrence
  - Count
  - Ratio
- Quotes Overlap
- Grammar Check
  - Spelling: pyspellchecker

### Model Architecture

I plan to construct a model with the architecture depicted in the following diagram. For the input to Deberta (`text`), I will pre-process by correcting any misspellings. In other aspects of feature engineering, I will utilize the `text` as is.

### References

- https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/discussion/428941

### My previous notebooks

- https://www.kaggle.com/code/tsunotsuno/debertav3-baseline-content-and-wording-models
- https://www.kaggle.com/code/tsunotsuno/debertav3-w-prompt-title-question-fields
- https://www.kaggle.com/code/tsunotsuno/debertav3-with-llama2-example
- https://www.kaggle.com/code/tsunotsuno/debertav3-lgbm-with-feature-engineering


In [7]:
if not is_colab :
  !pip install "/kaggle/input/autocorrect/autocorrect-2.6.1.tar"
  !pip install "/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

In [8]:
from typing import List
import numpy as np
import pandas as pd
import warnings
import logging
import os
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold, GroupKFold
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
import spacy
import re
from autocorrect import Speller
from spellchecker import SpellChecker
import lightgbm as lgb

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
disable_progress_bar()
tqdm.pandas()

In [9]:
import multiprocessing
cpu_num = multiprocessing.cpu_count()
print("cpu=", cpu_num)

cpu= 8


In [10]:
seed_base = 42

In [11]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=seed_base)

In [12]:
class CFG:
    model_name="another-bert"
    learning_rate=1.5e-5
    weight_decay=0.02
    hidden_dropout_prob=0.007
    attention_probs_dropout_prob=0.007
    num_train_epochs=5
    n_splits=4
    batch_size=3

    random_seed=seed_base
    save_steps=100
    max_length=512

## Dataload

In [13]:
DATA_DIR = "../input/commonlit-evaluate-student-summaries/"

prompts_train = pd.read_csv(DATA_DIR + "prompts_train.csv")
prompts_test = pd.read_csv(DATA_DIR + "prompts_test.csv")
summaries_train = pd.read_csv(DATA_DIR + "summaries_train.csv")
summaries_test = pd.read_csv(DATA_DIR + "summaries_test.csv")
sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")

## Preprocess

[Using features]

- Text Length
- Length Ratio
- Word Overlap
- N-grams Co-occurrence
  - count
  - ratio
- Quotes Overlap
- Grammar Check
  - spelling: pyspellchecker


In [14]:
class Preprocessor:
    def __init__(self,
                model_name: str,
                ) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(f"../input/{model_name}")
        self.twd = TreebankWordDetokenizer()
        self.STOP_WORDS = set(stopwords.words('english'))

        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.speller = Speller(lang='en')
        self.spellchecker = SpellChecker()

    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """
        def check_is_stop_word(word):
            return word in self.STOP_WORDS

        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))

    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int) -> int:
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)
        return len(common_ngrams)

    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)

        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))

        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}


    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):

        wordlist=text.split()
        amount_miss = len(list(self.spellchecker.unknown(wordlist)))

        return amount_miss

    def add_spelling_dictionary(self, tokens: List[str]) -> List[str]:
        """dictionary update for pyspell checker and autocorrect"""
        self.spellchecker.word_frequency.load_words(tokens)
        self.speller.nlp_data.update({token:1000 for token in tokens})

    def run(self,
            prompts: pd.DataFrame,
            summaries:pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:

        # before merge preprocess
        prompts["prompt_length"] = prompts["prompt_text"].apply(
            lambda x: len(word_tokenize(x))
        )
        prompts["prompt_tokens"] = prompts["prompt_text"].apply(
            lambda x: word_tokenize(x)
        )

        summaries["summary_length"] = summaries["text"].apply(
            lambda x: len(word_tokenize(x))
        )
        summaries["summary_tokens"] = summaries["text"].apply(
            lambda x: word_tokenize(x)
        )

        # Add prompt tokens into spelling checker dictionary
        prompts["prompt_tokens"].apply(
            lambda x: self.add_spelling_dictionary(x)
        )

#         from IPython.core.debugger import Pdb; Pdb().set_trace()
        # fix misspelling
        summaries["fixed_summary_text"] = summaries["text"].progress_apply(
            lambda x: self.speller(x)
        )

        # count misspelling
        summaries["splling_err_num"] = summaries["text"].progress_apply(self.spelling)

        # merge prompts and summaries
        input_df = summaries.merge(prompts, how="left", on="prompt_id")

        # after merge preprocess
        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']

        input_df['word_overlap_count'] = input_df.progress_apply(self.word_overlap_count, axis=1)
        input_df['bigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence,args=(2,), axis=1
        )
        input_df['bigram_overlap_ratio'] = input_df['bigram_overlap_count'] / (input_df['summary_length'] - 1)

        input_df['trigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )
        input_df['trigram_overlap_ratio'] = input_df['trigram_overlap_count'] / (input_df['summary_length'] - 2)

        input_df['quotes_count'] = input_df.progress_apply(self.quotes_count, axis=1)

        return input_df.drop(columns=["summary_tokens", "prompt_tokens"])

preprocessor = Preprocessor(model_name=CFG.model_name)

In [15]:
train = preprocessor.run(prompts_train, summaries_train, mode="train")
test = preprocessor.run(prompts_test, summaries_test, mode="test")

train.head()

100%|██████████| 4/4 [00:00<00:00, 4071.15it/s]


,student_id,prompt_id,text,content,wording,summary_length,fixed_summary_text,splling_err_num,prompt_question,prompt_title,prompt_text,prompt_length,length_ratio,word_overlap_count,bigram_overlap_count,bigram_overlap_ratio,trigram_overlap_count,trigram_overlap_ratio,quotes_count
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,64,The third wave was an experimental see how peo...,5,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,660,0.096970,14,4,0.063492,0,0.000000,0
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,54,They would rub it up with soda to make the sme...,2,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",1076,0.050186,18,22,0.415094,10,0.192308,0
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,269,"In Egypt, there were many occupations and soci...",32,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,625,0.430400,22,52,0.194030,23,0.086142,2
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,28,The highest class was Pharaohs these people we...,5,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,625,0.044800,6,6,0.222222,5,0.192308,0
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,232,The Third Wave developed rapidly because the ...,29,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,660,0.351515,23,27,0.116883,5,0.021739,4


In [16]:
import pickle
with open('pickled.pkl', 'wb') as f:
    pickle.dump(train, f)
    pickle.dump(test, f)

In [17]:
import pickle
with open('pickled.pkl', 'rb') as f:
    train = pickle.load(f)
    test = pickle.load(f)


In [18]:
# train = train[:512]

In [19]:
gkf = GroupKFold(n_splits=CFG.n_splits)

for i, (_, val_index) in enumerate(gkf.split(train, groups=train["prompt_id"])):
    train.loc[val_index, "fold"] = i

train.head()

,student_id,prompt_id,text,content,wording,summary_length,fixed_summary_text,splling_err_num,prompt_question,prompt_title,prompt_text,prompt_length,length_ratio,word_overlap_count,bigram_overlap_count,bigram_overlap_ratio,trigram_overlap_count,trigram_overlap_ratio,quotes_count,fold
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,64,The third wave was an experimental see how peo...,5,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,660,0.096970,14,4,0.063492,0,0.000000,0,3.0
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,54,They would rub it up with soda to make the sme...,2,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",1076,0.050186,18,22,0.415094,10,0.192308,0,2.0
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,269,"In Egypt, there were many occupations and soci...",32,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,625,0.430400,22,52,0.194030,23,0.086142,2,1.0
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,28,The highest class was Pharaohs these people we...,5,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,625,0.044800,6,6,0.222222,5,0.192308,0,1.0
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,232,The Third Wave developed rapidly because the ...,29,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...,660,0.351515,23,27,0.116883,5,0.021739,4,3.0


## Model Function Definition

In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)

    return (content_score + wording_score)/2

## Deberta Regressor

In [21]:
class ContentScoreRegressor:
    def __init__(self,
                model_name: str,
                model_dir: str,
                target: str,
                hidden_dropout_prob: float,
                attention_probs_dropout_prob: float,
                max_length: int,
                ):
        self.inputs = ["prompt_text", "prompt_title", "prompt_question", "fixed_summary_text"]
        self.input_col = "input"

        self.text_cols = [self.input_col]
        self.target = target
        self.target_cols = [target]

        self.model_name = model_name
        self.model_dir = model_dir
        self.max_length = max_length

        self.tokenizer = AutoTokenizer.from_pretrained(f"../input/{model_name}")
        self.model_config = AutoConfig.from_pretrained(f"../input/{model_name}")

        self.model_config.update({
            "hidden_dropout_prob": hidden_dropout_prob,
            "attention_probs_dropout_prob": attention_probs_dropout_prob,
            "num_labels": 1,
            "problem_type": "regression",
        })

        seed_everything(seed=42)

        self.data_collator = DataCollatorWithPadding(
            tokenizer=self.tokenizer
        )


    def tokenize_function(self, examples: pd.DataFrame):
        labels = [examples[self.target]]
        tokenized = self.tokenizer(examples[self.input_col],
                         padding=False,
                         truncation=True,
                         max_length=self.max_length)
        return {
            **tokenized,
            "labels": labels,
        }

    def tokenize_function_test(self, examples: pd.DataFrame):
        tokenized = self.tokenizer(examples[self.input_col],
                         padding=False,
                         truncation=True,
                         max_length=self.max_length)
        return tokenized

    def train(self,
            fold: int,
            train_df: pd.DataFrame,
            valid_df: pd.DataFrame,
            batch_size: int,
            learning_rate: float,
            weight_decay: float,
            num_train_epochs: float,
            save_steps: int,
        ) -> None:
        """fine-tuning"""

        sep = self.tokenizer.sep_token
        train_df[self.input_col] = (
                    train_df["prompt_title"] + sep
                    + train_df["prompt_question"] + sep
                    + train_df['splling_err_num'].astype(str) + " misspellings" + sep
                    + train_df["fixed_summary_text"]
                  )

        valid_df[self.input_col] = (
                    valid_df["prompt_title"] + sep
                    + valid_df["prompt_question"] + sep
                    + valid_df['splling_err_num'].astype(str) + " misspellings" + sep
                    + valid_df["fixed_summary_text"]
                  )

        train_df = train_df[[self.input_col] + self.target_cols]
        valid_df = valid_df[[self.input_col] + self.target_cols]

        model_content = AutoModelForSequenceClassification.from_pretrained(
            f"../input/{self.model_name}",
            config=self.model_config,
            ignore_mismatched_sizes=True
        )
        top_half_layer_freeze(model_content)

        train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
        val_dataset = Dataset.from_pandas(valid_df, preserve_index=False)

        train_tokenized_datasets = train_dataset.map(self.tokenize_function, batched=False)
        val_tokenized_datasets = val_dataset.map(self.tokenize_function, batched=False)

        # eg. "bert/fold_0/"
        model_fold_dir = os.path.join(self.model_dir, str(fold))


        training_args = TrainingArguments(
            output_dir=model_fold_dir,
            load_best_model_at_end=True, # select best model
            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=4,
            num_train_epochs=num_train_epochs,
            weight_decay=weight_decay,
            report_to='none',
            greater_is_better=False,
            save_strategy="steps",
            evaluation_strategy="steps",
            eval_steps=save_steps,
            save_steps=save_steps,
            metric_for_best_model="rmse",
            save_total_limit=1,
            fp16=True,
            warmup_steps = 100,
            gradient_accumulation_steps = 4,
            dataloader_num_workers = cpu_num,
            )

        trainer = Trainer(
            model=model_content,
            args=training_args,
            train_dataset=train_tokenized_datasets,
            eval_dataset=val_tokenized_datasets,
            tokenizer=self.tokenizer,
            compute_metrics=compute_metrics,
            data_collator=self.data_collator
        )

        trainer.train()

        shutil.rmtree(self.model_dir)

        model_content.save_pretrained(self.model_dir)
        self.tokenizer.save_pretrained(self.model_dir)


    def predict(self,
                test_df: pd.DataFrame,
                fold: int,
               ):
        """predict content score"""

        sep = self.tokenizer.sep_token
        in_text = (
                    test_df["prompt_title"] + sep
                    + test_df["prompt_question"] + sep
                    + test_df['splling_err_num'].astype(str) + " misspellings" + sep
                    + test_df["fixed_summary_text"]
                  )
        test_df[self.input_col] = in_text

        test_ = test_df[[self.input_col]]

        test_dataset = Dataset.from_pandas(test_, preserve_index=False)
        test_tokenized_dataset = test_dataset.map(self.tokenize_function_test, batched=False)

        model_content = AutoModelForSequenceClassification.from_pretrained(f"{self.model_dir}")
        model_content.eval()

        # e.g. "bert/fold_0/"
        model_fold_dir = os.path.join(self.model_dir, str(fold))

        test_args = TrainingArguments(
            output_dir=model_fold_dir,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = 4,
            dataloader_drop_last = False,
        )

        # init trainer
        infer_content = Trainer(
                      model = model_content,
                      tokenizer=self.tokenizer,
                      data_collator=self.data_collator,
                      args = test_args)

        preds = infer_content.predict(test_tokenized_dataset)[0]

        return preds

In [22]:
def train_by_fold(
        train_df: pd.DataFrame,
        model_name: str,
        target:str,
        save_each_model: bool,
        n_splits: int,
        batch_size: int,
        learning_rate: int,
        hidden_dropout_prob: float,
        attention_probs_dropout_prob: float,
        weight_decay: float,
        num_train_epochs: int,
        save_steps: int,
        max_length:int
    ):

    # delete old model files
    if os.path.exists(model_name):
        shutil.rmtree(model_name)

    os.mkdir(model_name)

    for fold in range(CFG.n_splits):
        print(f"fold {fold}:")

        train_data = train_df[train_df["fold"] != fold]
        valid_data = train_df[train_df["fold"] == fold]

        if save_each_model == True:
            model_dir =  f"{target}/{model_name}/fold_{fold}"
        else:
            model_dir =  f"{model_name}/fold_{fold}"

        csr = ContentScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir,
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )

        csr.train(
            fold=fold,
            train_df=train_data,
            valid_df=valid_data,
            batch_size=batch_size,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            num_train_epochs=num_train_epochs,
            save_steps=save_steps,
        )

def validate(
    train_df: pd.DataFrame,
    target:str,
    save_each_model: bool,
    model_name: str,
    hidden_dropout_prob: float,
    attention_probs_dropout_prob: float,
    max_length : int
    ) -> pd.DataFrame:
    """predict oof data"""
    for fold in range(CFG.n_splits):
        print(f"fold {fold}:")

        valid_data = train_df[train_df["fold"] == fold]

        if save_each_model == True:
            model_dir =  f"{target}/{model_name}/fold_{fold}"
        else:
            model_dir =  f"{model_name}/fold_{fold}"

        csr = ContentScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir,
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )

        pred = csr.predict(
            test_df=valid_data,
            fold=fold
        )

        train_df.loc[valid_data.index, f"{target}_pred"] = pred

    return train_df

def predict(
    test_df: pd.DataFrame,
    target:str,
    save_each_model: bool,
    model_name: str,
    hidden_dropout_prob: float,
    attention_probs_dropout_prob: float,
    max_length : int
    ):
    """predict using mean folds"""

    for fold in range(CFG.n_splits):
        print(f"fold {fold}:")

        if save_each_model == True:
            model_dir =  f"{target}/{model_name}/fold_{fold}"
        else:
            model_dir =  f"{model_name}/fold_{fold}"

        csr = ContentScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir,
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )

        pred = csr.predict(
            test_df=test_df,
            fold=fold
        )

        test_df[f"{target}_pred_{fold}"] = pred

    test_df[f"{target}"] = test_df[[f"{target}_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)

    return test_df

In [23]:
!mkdir content wording
for target in ["content", "wording"]:
    train_by_fold(
        train,
        model_name=CFG.model_name,
        save_each_model=True,
        target=target,
        learning_rate=CFG.learning_rate,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        weight_decay=CFG.weight_decay,
        num_train_epochs=CFG.num_train_epochs,
        n_splits=CFG.n_splits,
        batch_size=CFG.batch_size,
        save_steps=CFG.save_steps,
        max_length=CFG.max_length
    )

fold 0:


Step,Training Loss,Validation Loss,Rmse
100,No log,0.457434,0.676338
200,No log,0.409868,0.640210
300,No log,0.217407,0.466269
400,No log,0.155083,0.393806
500,0.318600,0.528165,0.726750
600,0.318600,0.242365,0.492306
700,0.318600,0.184758,0.429835
800,0.318600,0.239193,0.489074
900,0.318600,0.231287,0.480923
1000,0.151300,0.296447,0.544469


fold 1:


Step,Training Loss,Validation Loss,Rmse
100,No log,0.272620,0.522130
200,No log,0.316640,0.562708
300,No log,0.330209,0.574638
400,No log,0.369248,0.607658
500,0.290900,0.338661,0.581946
600,0.290900,0.317887,0.563815
700,0.290900,0.320549,0.566171
800,0.290900,0.375114,0.612466
900,0.290900,0.257973,0.507911
1000,0.149600,0.240845,0.490760


fold 2:


...

fold 2:


Step,Training Loss,Validation Loss,Rmse
100,No log,0.450144,0.670928
200,No log,0.436635,0.660784
300,No log,0.485680,0.696908
400,No log,0.374336,0.611830
500,0.496500,0.370029,0.608300
600,0.496500,0.258111,0.508046
700,0.496500,0.298619,0.546460
800,0.496500,0.268494,0.518164
900,0.496500,0.316173,0.562292
1000,0.277000,0.308482,0.555411


fold 3:


Step,Training Loss,Validation Loss,Rmse
100,No log,0.830321,0.911220
200,No log,0.506558,0.711729
300,No log,0.650758,0.806696
400,No log,0.468338,0.684352
500,0.433800,0.412543,0.642295
600,0.433800,0.708334,0.841626
700,0.433800,0.548637,0.740700
800,0.433800,0.454153,0.673909
900,0.433800,0.450678,0.671325
1000,0.255000,0.438390,0.662110


Step,Training Loss,Validation Loss,Rmse
100,No log,0.830321,0.911220
200,No log,0.506558,0.711729
300,No log,0.650758,0.806696
400,No log,0.468338,0.684352
500,0.433800,0.412543,0.642295
600,0.433800,0.708334,0.841626
700,0.433800,0.548637,0.740700
800,0.433800,0.454153,0.673909
900,0.433800,0.450678,0.671325
1000,0.255000,0.438390,0.662110


In [ ]:
# !cp -r . /content/drive/MyDrive/Kaggle/commitlit

In [24]:
if is_colab:
  import json
  from kaggle.api.kaggle_api_extended import KaggleApi

  def dataset_upload():
      global USERID, EX_NO, UPLOAD_DIR
      id = f'{USERID}/{EX_NO}'
      dataset_metadata = {}
      dataset_metadata['id'] = id
      dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
      dataset_metadata['title'] = f'{TITLE}'
      with open(UPLOAD_DIR + 'dataset-metadata.json', 'w') as f:
          json.dump(dataset_metadata, f, indent=4)
      api = KaggleApi()
      api.authenticate()
      # データセットがない場合
      if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
          api.dataset_create_new(folder=UPLOAD_DIR,
                                convert_to_csv=False,
                                dir_mode='skip'
                                #dir_mode='zip'
                                )
      # データセットがある場合
      else:
          api.dataset_create_version(folder=UPLOAD_DIR,
                                    version_notes='update',
                                    convert_to_csv=False,
                                    delete_old_versions=True,
                                    dir_mode='skip'
                                    #dir_mode='zip'
                                    )

In [25]:
%%time
if is_colab:
  !mkdir tmp
  !zip -r ./tmp/output content wording pickled.pkl
  # !zip -r ./tmp/output  pickled.pkl

  adding: content/ (stored 0%)
  adding: content/another-bert/ (stored 0%)
  adding: content/another-bert/fold_0/ (stored 0%)
  adding: content/another-bert/fold_0/config.json (deflated 53%)
  adding: content/another-bert/fold_0/tokenizer.json (deflated 77%)
  adding: content/another-bert/fold_0/added_tokens.json (deflated 33%)
  adding: content/another-bert/fold_0/spm.model (deflated 50%)
  adding: content/another-bert/fold_0/pytorch_model.bin (deflated 21%)
  adding: content/another-bert/fold_0/special_tokens_map.json (deflated 54%)
  adding: content/another-bert/fold_0/tokenizer_config.json (deflated 74%)
  adding: content/another-bert/fold_1/ (stored 0%)
  adding: content/another-bert/fold_1/config.json (deflated 53%)
  adding: content/another-bert/fold_1/tokenizer.json (deflated 77%)
  adding: content/another-bert/fold_1/added_tokens.json (deflated 33%)
  adding: content/another-bert/fold_1/spm.model (deflated 50%)
  adding: content/another-bert/fold_1/pytorch_model.bin (deflated 

In [ ]:
%%time
if is_colab:
  TITLE = 'commitlit-deberta-v3-large-misspel2'
  EX_NO = 'commitlit-deberta-v3-large-misspel2'  # 実験番号などを入れる、folderのpathにする
  USERID = 'aruaru0'
  UPLOAD_DIR = './tmp/'
  dataset_upload()

Starting upload for file output.zip


100%|██████████| 10.2G/10.2G [07:37<00:00, 24.0MB/s]


Upload successful: output.zip (10GB)
CPU times: user 14.6 s, sys: 5.48 s, total: 20.1 s
Wall time: 7min 40s


In [ ]:
# !cp -r ./tmp /content/drive/MyDrive/Kaggle/commitlit

## terminate

In [ ]:
if is_colab:
  from google.colab import runtime
  runtime.unassign()

In [ ]:
for target in ["content", "wording"]:
    train = validate(
        train,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

    rmse = mean_squared_error(train[target], train[f"{target}_pred"], squared=False)
    print(f"cv {target} rmse: {rmse}")



In [ ]:
for target in ["content", "wording"]:
    test = predict(
        test,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

In [ ]:
train.head()

## upload dataset

In [ ]:
# %%time
# if is_colab:
#   !mkdir tmp
#   !zip -r ./tmp/output content wording pickled.pkl

In [ ]:
# %%time
# if is_colab:
#   TITLE = 'devert-v3-trained'
#   EX_NO = 'commitlit-2023'  # 実験番号などを入れる、folderのpathにする
#   USERID = 'aruaru0'
#   UPLOAD_DIR = './tmp/'
#   dataset_upload()

## LGBM model

In [ ]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text", "fixed_summary_text",
                "prompt_question", "prompt_title",
                "prompt_text"
               ] + targets

In [ ]:
model_dict = {}

for target in targets:
    models = []

    for fold in range(CFG.n_splits):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
            'boosting_type': 'gbdt',
            'random_state': 42,
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.048,
            'max_depth': 3,
            'lambda_l1': 0.0,
            'lambda_l2': 0.011
        }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                               lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)

    model_dict[target] = models

In [ ]:
import pickle
with open('lgbm.pkl', 'wb') as f:
    pickle.dump(model_dict, f)

In [ ]:
%%time
if is_colab:
  !mkdir tmp
  !zip -r ./tmp/output content wording pickled.pkl lgbm.pkl

In [ ]:
%%time
if is_colab:
  TITLE = 'roberta-base-trained'
  EX_NO = 'roberta-base-trained'  # 実験番号などを入れる、folderのpathにする
  USERID = 'aruaru0'
  UPLOAD_DIR = './tmp/'
  dataset_upload()

## CV Score

In [ ]:
# cv
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []

    for fold, model in enumerate(models):
        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)

    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

## Predict

In [ ]:
drop_columns = [
                #"fold",
                "student_id", "prompt_id", "text", "fixed_summary_text",
                "prompt_question", "prompt_title",
                "prompt_text",
                "input"
               ] + [
                f"content_pred_{i}" for i in range(CFG.n_splits)
                ] + [
                f"wording_pred_{i}" for i in range(CFG.n_splits)
                ]

In [ ]:
pred_dict = {}
for target in targets:
    models = model_dict[target]
    preds = []

    for fold, model in enumerate(models):
        X_eval_cv = test.drop(columns=drop_columns)

        pred = model.predict(X_eval_cv)
        preds.append(pred)

    pred_dict[target] = preds

In [ ]:
for target in targets:
    preds = pred_dict[target]
    for i, pred in enumerate(preds):
        test[f"{target}_pred_{i}"] = pred

    test[target] = test[[f"{target}_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)

In [ ]:
test

## Create Submission file

In [ ]:
sample_submission

In [ ]:
test[["student_id", "content", "wording"]]

In [ ]:
test[["student_id", "content", "wording"]].to_csv("submission.csv", index=False)

## Summary

CV result is like this.

| | content rmse |wording rmse | mcrmse | LB| |
| -- | -- | -- | -- | -- | -- |
|baseline| 0.494 | 0.630 | 0.562 | 0.509 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-baseline-content-and-wording-models)|
| use title and question field | 0.476| 0.619 | 0.548 | 0.508 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-w-prompt-title-question-fields) |
| Debertav3 + LGBM | 0.451 | 0.591 | 0.521 | 0.461 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-lgbm-with-feature-engineering) |
| Debertav3 + LGBM with spell autocorrect | 0.448 | 0.581 | 0.514 | 0.459 |nogawanogawa's original code
| Debertav3 + LGBM with spell autocorrect and tuning | 0.442 | 0.566 | 0.504 | 0.453 | this notebook |

The CV values improved slightly, and the LB value is improved.

In [ ]:
if is_colab:
  from google.colab import runtime
  runtime.unassign()